<a href="https://colab.research.google.com/github/mcnica89/MATH4060/blob/main/FinalProject_Cant_Stop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import itertools
import jax.numpy as jnp
from jax import random as jrandom
from jax import nn as jnn
from jax import jit
import numpy as np
import random
import time
import sys
import jax

In [2]:
'''Implementation of the board game "CANT STOP"

How the game is represented in Python:

-----Game parameters (that do not change while the game is being played)

N_PLAYERS
   A postive integer
  This is the number of players playing
  In the classic game rules, N_Players = 4

N_COL_TO_WIN
   A positive integer
   This is the number of columns you need to claim to win the game
   In the classic game rules, N_Col_To_Win = 3

N_MAX_RUNNERS
   A positive integer
   This is the maximum number of runners you can have
   In the classic game rules, N_Max_Runners = 3

PLAYER_COL_STATE_INIT
   An vector of shape (11,) of non-negative integers
   This is the number of squares in each game column
   In the classic game rules, this is [3,5,7,9,11,13,11,9,7,5,3]

NOTE on column labelling:
   In the game, the columns are labeled 2-12 (corresponding to dice rolls)
   In Python, the columns locations are indexed 0-10
   This means that to translate from column in Python to columns in the game,
   one must often add or subtract 2 from the column indices. 

---Variables: (that represent what is going on in the game as it is played)

active_player_index
   An index from the range [0,N_PLAYERS-1] indicating whose turn it current is

player_col_state
   An array of shape (N_players,11) of integers
   Each row is the number of squares remaining for that player in each col
   NOTE: 
     This is the number of squares REMAINING, these start at PLAYER_COL_STATE_INIT
     and count DOWN to zero as the game progresses. When the get to zero, the player has claimed the column
   WARNING:
     We will not prohibit these from being negative even though it doesn't mean anything in the game
     (This can happen if the player goes past the number needed to claim the column)

illegal_col
   A vector of shape (11,) of boolean
   Contains the information on which columns are still in play
   (columns that have been claimed by a player are not legal to play in anymore)

runner_col_state
  A vector of shape (11,) of non-negative integers
  Indicates the current state of how far the runners have advanced in each column
  A zero indicates that there is no runner in that column at all
  NOTES:
   1. count_nonzero(runner_col) should not exceed N_Max_Runners for legal runner states
   2. Since player_col_state counts DOWN to 0, runner_col is SUBTRACTED from player_col_state when the player chooses to stop rolling

dice_rolls
  A vector of shape (4,) of integers [1,6] indicating the outcome of the 4 dice rolls

runner_col_choices
  A vector of shape (N_choices, 11) of non-negative integers
  Indicates the available CHOICES the player has of where the runners could be
  This corresponds to legal choices for choosing pairings of the dice
  NOTES:
    1. By the rules of the game, N_choices can be at most 6
    2. If N_choices = 0, then this indicates that there are no legal moves and the player has busted

roll_again
  A boolean on whether or not the player wants to rolls again'''

'Implementation of the board game "CANT STOP"\n\nHow the game is represented in Python:\n\n-----Game parameters (that do not change while the game is being played)\n\nN_PLAYERS\n   A postive integer\n  This is the number of players playing\n  In the classic game rules, N_Players = 4\n\nN_COL_TO_WIN\n   A positive integer\n   This is the number of columns you need to claim to win the game\n   In the classic game rules, N_Col_To_Win = 3\n\nN_MAX_RUNNERS\n   A positive integer\n   This is the maximum number of runners you can have\n   In the classic game rules, N_Max_Runners = 3\n\nPLAYER_COL_STATE_INIT\n   An vector of shape (11,) of non-negative integers\n   This is the number of squares in each game column\n   In the classic game rules, this is [3,5,7,9,11,13,11,9,7,5,3]\n\nNOTE on column labelling:\n   In the game, the columns are labeled 2-12 (corresponding to dice rolls)\n   In Python, the columns locations are indexed 0-10\n   This means that to translate from column in Python to c

# Helper functions

## Dice Functions
Note that we will compress the four 6-sided dice rolls down to an integer in the range $[0,1295]$ which we call the `diceNum`. This is done by thinking of the roll as a 4 digit number in base 6.

In [3]:
def diceRoll_to_diceNum(diceRollArray):
  '''Converts an array of shape (4,) of the four 6 sided dice to the diceNum in [0,1295]'''
  #Note: Dice rolls are assumed to be numbers 1-6 (i.e. they start at 1!)
  powersOfSix = jnp.array([1,6,36,216])
  return jnp.inner(powersOfSix, jnp.array(diceRollArray)-1)

def diceNum_to_diceRoll(diceNum):
  '''Converts the diceNum in [0,1295] to an array of an array of shape (4,) of the 4 dice rolls'''
  #Note: Dice rolls are assumed to be numbers 1-6 (i.e. they start at 1!)
  powersOfSix = jnp.array([1,6,36,216])

  return 1+(diceNum // powersOfSix) % 6

def conversionTests():
  assert( jnp.all(diceNum_to_diceRoll(diceRoll_to_diceNum(jnp.array([2,1,1,1]))) == jnp.array([2,1,1,1])) )
  assert(diceRoll_to_diceNum(diceNum_to_diceRoll(999)) == 999)
  return True

conversionTests()

True

In [4]:
def calculate_runnerDicePairArray():
  '''Returns an array of shape (6,11,1296) which contains the possible runner locations (encoded as one hot (11,) vectors)
   for all the possible 1296 dice rolls, and 6 possible ways to pair the dice'''
  #Input:
  # Nothing!
  #Output:
  #  a boolean vector of size (6,11,1296) so that out[i,j,:] is the (11,) boolean vector of the runner locations
  #  when the dice roll #i is rolled and choice j is selected
  #  This precomputed output is used when computing the runners that can occur in the game

  #Create an array of shape (4,6,6,6,6) that contains all possible dice rolls
  #  i.e. the entry [:,a,b,c,d] = [a,b,c,d] is 4 dice rolls and a,b,c,d all run from 0 to 5
  four_dice_indices = jnp.indices((6,6,6,6)) 
 
  #Create an array with all 6 ways to choose 2 out of 4 dice
  #  Pairing 0 = choose dice 1 and dice 2
  #  Pairing 1 = choose dice 1 and dice 3 
  #  ... 
  #  Pairing 5 = choose dice 3 and dice 4
  pairing = jnp.array([[1,1,0,0],[1,0,1,0],[1,0,0,1],[0,1,1,0],[0,1,0,1],[0,0,1,1]])

  #Create an array of shape (6,6,6,6,6) which gives the value of the pairing 
  #  i.e. the (a,b,c,d,p) entry is the value of pairing p when the dice come up a,b,c,d
  four_dice_pairings = jnp.einsum("iabcd,pi->abcdp",four_dice_indices,pairing)
  
  #The same array, but of shape (6,6,6,6,6,11) now where it has been converted to a one hot encoding
  #  i.e. (a,b,c,d,p:) is an array of shape (11,) with the one hot encoding of the pairing
  four_dice_pairings_one_hot = jnn.one_hot(four_dice_pairings,11)
  flattened_but_out_of_order = jnp.reshape(four_dice_pairings_one_hot,(1296,6,11),order='F')
  return jnp.transpose( flattened_but_out_of_order, (1,2,0)) #put them in the desired order!
  

In [5]:
#DiceArray tests
DicePairArray = calculate_runnerDicePairArray()
print(jnp.shape(DicePairArray))
N=diceRoll_to_diceNum([1,2,3,6])
print(diceNum_to_diceRoll(N))
print(DicePairArray[:,:,N])

(6, 11, 1296)
[1 2 3 6]
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


## Helper functions for dealing with runners

In [6]:
@jit
def calculate_player_N_col_claimed(player_col_state):
  ''' Calculate player "scores" (i.e. number of columns claimed) from the board state'''
  #Input:
  #  player_col_state = An int array of size (N_players, 11) showing how many entries REMAINING until column is claimed for each player
  #Output: 
  #  An int vector of size (N_players,) showing how many columns each player has claimed. (In normal rules, first to 3 columns wins) 
  return  jnp.count_nonzero(player_col_state <= 0, axis=1)

@jit
def calculate_illegal_col(player_col_state):
  '''Calculate which columns are legal from the board state (i.e. the unclaimed columns)''' 
  #Input:
  #  player_col_state = An int array of size (N_players, 11) showing how many entries REMAINING until column is claimed for each player
  #Output: 
  #  An boolean vector of size (11,) showing which columns are legal 
  return jnp.any(player_col_state <= 0, axis=0)

def calculate_col_tests():
  '''Test the calculate columns helper functions'''
  players_col_state = jnp.array([[2,3,4,5,6,7,8,9,10,11,12],[2,0,4,5,6,7,8,9,10,11,12],[2,3,0,0,0,7,8,9,10,11,12]])

  players_N_col_claimed = calculate_player_N_col_claimed(players_col_state)
  assert jnp.all(players_N_col_claimed == jnp.array([0,1,3]))

  illegal_col = calculate_illegal_col(players_col_state)
  assert jnp.all(illegal_col == jnp.array([0,1,1,1,1,0,0,0,0,0,0]))

  return True

assert calculate_col_tests()

In [7]:
@jit
def are_runners_legal(runner_col_states, illegal_col, N_MAX_RUNNERS=3):
  '''Checks if a batch of runner states are legal or not'''
  #Input:
  #  runner_col_states = an int vector of size (N,11) of runner positions
  #  illegal_col = a boolean vector of size (11,) with which columns are illegal
  #Output:
  #  a boolean vector of size (N,) with which of the runner_col_states are legal

  #Number of runners is legal iff there are <=N_MAX_RUNERS runners active:
  are_number_of_runners_legal = (jnp.count_nonzero(runner_col_states,axis=1) <= N_MAX_RUNNERS)

  #Check if all the runners are in legal columns
  #  In each column, either illegal_column must be 0 OR runners must be 0
  illegal_col_or_runner_is_0 = jnp.logical_or(runner_col_states == 0, illegal_col == False)
  #  This must happen in every single column
  are_runners_in_legal_col = jnp.all(illegal_col_or_runner_is_0,axis=1) 
  
  return jnp.logical_and(are_number_of_runners_legal,are_runners_in_legal_col)

def are_runners_legal_tests():
  '''Tests for the are_runners_legal function'''
  
  illegal_col = jnp.array([0,1,0,1,0,1,0,1,0,1,0],dtype=jnp.dtype('b'))
  runner_col_states = jnp.array([[1,1,0,0,0,0,0,0,0,0,0],[1,1,1,1,0,0,0,0,0,0,0],[1,0,1,0,1,0,1,0,0,0,0],[1,0,1,0,1,0,0,0,0,0,0]],dtype=jnp.dtype('u1'))
  N_MAX_RUNNERS = 3

  #In this test:
  #0: runner in illegal column -> not legal
  #1: runner in illegal column AND to many runners -> not legal
  #2: runner in all legal columns but too many runners -> not legal
  #3: runner in all legal columns and exactly 3 runners -> legal

  assert jnp.all(are_runners_legal(runner_col_states,illegal_col,N_MAX_RUNNERS) == jnp.array([False,False,False,True])) 
  return True

assert are_runners_legal_tests()

In [8]:
dice_num = jnp.array([1,2,3,4])
x =DicePairArray[0:3,:,dice_num]
print(jnp.shape(x))

(3, 11, 4)


In [9]:
DicePairArray = calculate_runnerDicePairArray()
@jit
def generate_all_choices_and_legality(dice_num,player_col_state,runner_col_state, N_MAX_RUNNERS=3): 
  illegal_col = calculate_illegal_col(player_col_state)
  #print("illegal_col", jnp.shape(illegal_col))
  '''Computes out ALL the possible moves based on the dice and 
  whether or not they are legal based on the current state and dice'''
  #  In this version, the input is the dice_num (which is a number in [0,1295]) and 
  #  the array DiceArray are assumed to exists
  #  which is caluclated with the DiceArray function
  #Calculate all the 9 possible moves of playing both pairs (i.e. double) and with any single pair
  # (We will work out which are legal moves afterwards!)

  #Use the dice_num to lookup the runner possibilities from DiceArray
  dice_sums_with_1_cols = DicePairArray[0:3,:,dice_num]
  dice_sums_without_1_cols = DicePairArray[5:2:-1,:,dice_num]

  #print("dice_sums_with_1_cols", jnp.shape(dice_sums_with_1_cols)) 
  #print("runner_col_state", jnp.shape(runner_col_state))
  #This 5:2:-1 gets the pairings in reverse order so that they are complentary to the pairings from dice_sums_with_1
  

  double_runner_choices = runner_col_state + dice_sums_with_1_cols + dice_sums_without_1_cols
  single_runner_choices_with_1 = runner_col_state + dice_sums_with_1_cols 
  single_runner_choices_without_1 = runner_col_state + dice_sums_without_1_cols

  #print("double_runner_choices",jnp.shape(double_runner_choices))

  #Compute if the choices with both pairing played (i.e. double) are legal
  are_double_runners_legal = are_runners_legal(double_runner_choices,illegal_col, N_MAX_RUNNERS)
  #print("are_double_runners_legal",jnp.shape(are_double_runners_legal))
  #print(are_double_runners_legal)
  are_double_runners_illegal = jnp.logical_not(are_double_runners_legal)


  #The moves with a single pair are only legal if the corresponding move with both pairs is illegal 
  #  (i.e. its legal to play only one pair iff after you play it, playing the next move is not legal)
  #  This means we can compute if they are legal on their own first and then
  #  logical_and it with the double runners

  #  first check if they would be ok on their own.
  are_single_runners_with_1_ok = are_runners_legal(single_runner_choices_with_1,illegal_col, N_MAX_RUNNERS)
  are_single_runners_without_1_ok = are_runners_legal(single_runner_choices_without_1,illegal_col, N_MAX_RUNNERS)

  #  then we logical and it with the double runners to only legalize these moves if playing both was illegal
  are_single_runners_with_1_legal = jnp.logical_and(are_double_runners_illegal,are_single_runners_with_1_ok)
  are_single_runners_without_1_legal = jnp.logical_and(are_double_runners_illegal,are_single_runners_without_1_ok)
  #print("ok!")
  #Combine everything together to be outputed
  #all_runner_choices = jnp.row_stack()
  all_runner_choices = jnp.row_stack([double_runner_choices,single_runner_choices_with_1,single_runner_choices_without_1]) 
  #print("all_runner_choices", jnp.shape(all_runner_choices))
  all_runner_choices_legal = jnp.concatenate([are_double_runners_legal, are_single_runners_with_1_legal, are_single_runners_without_1_legal])
  #print("all_runner_choices_legal", jnp.shape(all_runner_choices_legal))

  return all_runner_choices, all_runner_choices_legal

def generate_all_choices_and_legality_tests():
  dice_rolls = jnp.array([1,2,3,4],dtype=jnp.dtype('u1'))
  dice_num = diceRoll_to_diceNum(dice_rolls)
  print(dice_num)
  runner_col_state = jnp.array([0,0,0,0,0,0,0,0,0,0,0],dtype=jnp.dtype('u1'))
  illegal_col = jnp.array([0,1,0,0,0,0,0,0,0,0,0],dtype=jnp.dtype('u1'))
  expected_answer = jnp.array([[0,0,0,0,0,1,0,0,0,0,0],[0,0,0,2,0,0,0,0,0,0,0],[0,0,1,0,1,0,0,0,0,0,0]], dtype=jnp.dtype('u1'))
  N_MAX_RUNNERS = 3

  actual_answer =  generate_all_choices_and_legality(dice_num,illegal_col,runner_col_state, N_MAX_RUNNERS)
  #print(actual_answer)
  return True


In [10]:
@jit
def update_player_col_state(active_player_index, player_col_state, runner_col_state):
  '''Move a players peices forward by the amount on the runners 
    (This is called when a player bank's their runners and ends their turn by choice)''' 
  #Input:
  #  active_player_index = index of whose turn it is
  #  player_col_state = int array of size (N_player, 11) with squares remaining in each column
  #  runner_col_state = int vector of size (11,) with runner locations
  #Output:
  #  An updated version of player_col_state where the positions have been moved up by the runners.

  #All we have to do is a subtraction, but ensure that we don't go below zero
  updated_active_player_col_state = jnp.clip(player_col_state[active_player_index] - runner_col_state, 0, None)
  #print("updated col state", jnp.shape(updated_active_player_col_state))
  #ans = player_col_state.at[active_player_index].set(updated_active_player_col_state)
  #print("ans", jnp.shape(ans))
  return player_col_state.at[active_player_index].set(updated_active_player_col_state)

def update_player_col_test():
  '''Test the update_player_col_state function'''
  player_col_state = jnp.array([[2,3,4,5,6,7,8,9,10,11,12],[2,0,4,5,6,7,8,9,10,11,12],[2,3,0,0,0,7,8,9,10,11,12]])
  runner_col_state = jnp.array([1,1,1,1,1,1,1,1,1,1,12])
  active_player_index = 1
  expected_answer = jnp.array([[2,3,4,5,6,7,8,9,10,11,12],[1,0,3,4,5,6,7,8,9,10,0],[2,3,0,0,0,7,8,9,10,11,12]])
  
  assert jnp.all(update_player_col_state(active_player_index, player_col_state, runner_col_state)==expected_answer)
  return True

assert update_player_col_test()

# Main game simulator

In [11]:
def random_true_index(key, bool_array):
  '''Given an boolean array, and a JAX random key, return a random index where True appears'''
  return jrandom.choice(key, jnp.arange(jnp.shape(bool_array)[0]) , p= bool_array/jnp.sum(bool_array))

def dummyAI(active_player_index, player_col_state, choices, legal, N_COL_TO_WIN = 3, N_MAX_RUNNERS = 3):
  '''A placeholder game AI that makes all choices randomly'''
  #Input:
  #  active_player_index = An int with whose turn it currently is (which player the AI is playing for)
  #  player_col_state = An int array of size (N_players, 11) showing how many entries REMAINING until column is claimed for each player
  #  choices = An array of size (9,11) with the 9 possible choices available to the player
  #  legal = An array of size (9,) with whether or not each of the 9 choices are legal
  #  N_Col_To_Win, N_Max_Runners = Integers that can specify variants of the game rules 
  #Output: A tupl (choice_index, roll_again)
  #  1st entry: choice_index = An integer in [0,8] with which choice is to be played
  #             (Note, you must make sure the index you play is legal!)
  #  2nd entry: roll_again = A boolean on whether or not to roll again 
  
  #To get the unique choices you can do jnp.unique(choices[legal==True],axis=0)
  key = jrandom.PRNGKey(int(time.time()))
  choice_index = random_true_index(key,legal) #choose a random legal option!
  roll_again = bool(random.randint(0,1)) #choose to roll again randomly!
  
  return choice_index, roll_again #dummy AI picks the first choice and stops rolling again

In [12]:
def simulate_game(Player_AI, Verbose = False, N_PLAYERS=2, N_COL_TO_WIN=5, N_MAX_RUNNERS=3, PLAYER_COL_STATE_INIT=[3,5,7,9,11,13,11,9,7,5,3]):
  '''Run a simulation of the game Can't Stop!'''
  #Input:
  #   randome_key = jrandom key used for dice rolls
  #   Player_AI = List with the functions for player AIs
  #   Verbose = whether or not to print out a play-by-play of the game
  #Output:
  #  An array of shape (N_players,) with a 1 at the player who won
  
  #Initialize game state
  player_col_state = jnp.tile(jnp.array(PLAYER_COL_STATE_INIT,dtype=jnp.dtype('i1')),(N_PLAYERS, 1))
  zero_runner = jnp.zeros( 11 ,dtype=jnp.dtype('u1')) 

  #This flag tells us when the game is over
  game_in_progress = True
  
  #Choose a random player to start
  #  Note that the actual player who starts is one player later than the one chose here.
  active_player_index = random.randint(0,N_PLAYERS-1) 


  #Main loop that goes until someone wins the game
  
  turn_number = 0
  while game_in_progress: #This will loop until the game ends 
    turn_number += 1
    #Update whose turn it is
    active_player_index = (active_player_index + 1) % N_PLAYERS
    
    if Verbose : print("Turn Number ",turn_number)
    if Verbose : print("Player ",active_player_index,":") 
    if Verbose : print("--Player Column State: \n",player_col_state)

    #Reset runners and "busted"/"roll again" flags that tell us the game state
    runner_col_state = zero_runner
    not_busted_state = True
    roll_again_state = True

    #Loop while player is chooising to rolling on their turn
    roll_iteration = 0
    while roll_again_state:
      roll_iteration += 1
      #This represents a random dice roll 
      # (1296 = 6**4 is the number of possibilities for 4 6-sided dice)
      dice_num =  random.randint(0,1295)

      if Verbose : print("----Roll: ",diceNum_to_diceRoll(dice_num))      
      
      #Generate all 9 possible runner choices and whether or not they are legal
      runner_choices, runner_legal = generate_all_choices_and_legality(dice_num,player_col_state,runner_col_state, N_MAX_RUNNERS)
      
      if Verbose : print("----Options:\n", jnp.unique(runner_choices[runner_legal==True],axis=0) ) 

      #Update the busted state: you are only not busted if you have at least one legal choice
      any_legal_choices = jnp.any(runner_legal)
      not_busted_state = not_busted_state and any_legal_choices
      if Verbose: print("---Busted? ", not not_busted_state)

      if not_busted_state:
        #Send the choices to the AI to choose from
        #Note that we run this even for ones where we've already busted (we just make sure to do nothing with this data in that case)
        active_player_AI = Player_AI[active_player_index]
        choice_index, roll_again_state = active_player_AI(active_player_index, player_col_state, runner_choices, runner_legal, N_COL_TO_WIN, N_MAX_RUNNERS)

        #Update the runner positions if you are not busted 
    
        runner_col_state = runner_choices[choice_index]
        
        if Verbose : print("----Runners chosen:\n", runner_col_state) 
        if Verbose :print("----Roll again choice: ", roll_again_state)
      
      #If you are busted you are not allowed to roll again! 
      roll_again_state = roll_again_state and not_busted_state
      

    #-----------------------
    #End of the players turn:
    #-----------------------

    #This resets your runners to zero if you had busted
    runner_col_state = runner_col_state*not_busted_state
    
    #Move the pieces according to the runner positions
    player_col_state = update_player_col_state(active_player_index,player_col_state,runner_col_state) 

    #Calculate columns claimed and check if game is over
    player_N_col_claimed = calculate_player_N_col_claimed(player_col_state)
    game_in_progress = not jnp.any(player_N_col_claimed >= N_COL_TO_WIN)

   
  #At the end of this loop, someone has won!
  if Verbose : 
    print("GAME OVER!") 
    print("Final board state:\n ", player_col_state)
    print("Final claimed column count: ", player_N_col_claimed)
    print("Winner: ", ( player_N_col_claimed >= N_COL_TO_WIN ))


  return ( player_N_col_claimed >= N_COL_TO_WIN )

#Play a bunch of games and record final result
def play_N_games(N_games, AIs, N_PLAYERS = 2):
  winners = jnp.zeros(N_PLAYERS,dtype=jnp.dtype('i4'))
  for i in range(N_games):
    if i % (N_games/10) == 0:
      print( int(100*(i / N_games)), "% done")
    winners += simulate_game(AIs)
  print("After ", N_games, " winners are ",winners)

In [13]:
simulate_game([dummyAI,dummyAI],True)

Turn Number  1
Player  0 :
--Player Column State: 
 [[ 3  5  7  9 11 13 11  9  7  5  3]
 [ 3  5  7  9 11 13 11  9  7  5  3]]
----Roll:  [2 4 3 6]
----Options:
 [[0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.]]
---Busted?  False
----Runners chosen:
 [0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
----Roll again choice:  False
Turn Number  2
Player  1 :
--Player Column State: 
 [[ 3  5  7  8 11 13 11  9  6  5  3]
 [ 3  5  7  9 11 13 11  9  7  5  3]]
----Roll:  [2 6 2 5]
----Options:
 [[0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]]
---Busted?  False
----Runners chosen:
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
----Roll again choice:  True
----Roll:  [1 5 2 3]
----Options:
 [[0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 2. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 2. 0. 0. 0. 0.]]
---Busted?  False
----Runners chosen:
 [0. 0. 1. 0. 0. 2. 1. 0. 0. 0. 0.]
----Roll again choice:  True
----Roll

DeviceArray([ True, False], dtype=bool)

In [14]:
%timeit simulate_game([dummyAI,dummyAI],False)

1 loop, best of 5: 268 ms per loop


In [15]:
play_N_games(100,[dummyAI,dummyAI])

0 % done
10 % done
20 % done
30 % done
40 % done
50 % done
60 % done
70 % done
80 % done
90 % done
After  100  winners are  [57 43]


# A simple feature based AI

## Some probabilities used in the AI

In [16]:
#This function also assumes that the DicePairArray global variable has been calculated already
@jit
def prob_to_miss_targets(targets):
  '''Compute the probability to miss a list of target cols'''
  #Input:
  #  targets = a boolean array of shape (11,) with which are targets
  #Output:
  #  A real number with the probability to miss all the targets from targets when rolling 4 dice and pairing them
  hit_target = jnp.einsum("abp,b->pa",DicePairArray,targets)
  any_hit_target = jnp.any(hit_target > 0, axis=1)

  #Count the number of times we get a hit!
  number_of_dice_rolls_that_hit_target = jnp.count_nonzero(any_hit_target)
  return (1296- number_of_dice_rolls_that_hit_target)/1296

def prob_to_miss_targets_tests():
  '''Test the prob_to_miss_targets_function'''
  #No 2's test:
  #Probability to miss getting a 2 somewhere
  p_func = prob_to_miss_targets(jnp.array([1,0,0,0,0,0,0,0,0,0,0]))

  #On the other hand you can only miss getting a 2 if you have either:
  # exactly 4 non-1's rolled in the dice roll (Probabolity (5/6)^4)
  # exactly 3 non-1's in the dice roll (Probability 4*(5/6)^3*(1/6))
  p_expected = (5/6)**4+4*(5/6)**3*(1/6)

  assert jnp.isclose(p_func,p_expected) 

  #No 3's test:
  #Probability to miss getting a 3 somewhere
  p_func = prob_to_miss_targets(jnp.array([0,1,0,0,0,0,0,0,0,0,0]))

  #On the other hand you can only get a 3 if you have at least one 1 and at least one 2
  # so you miss getting a 3 if you have both NO 1's or NO 2's in the roll
  # By inclusion exclision, the probability of NO 1's or NO 2's is:
  # P(NO 1's) + P(NO 2's) - P(NO 1's and NO 2's):
  # = (5/6)^4 + (5/6)^4 - (4/6)^4
  p_expected = (5/6)**4 + (5/6)**4 - (4/6)**4
   
  assert jnp.isclose(p_func,p_expected) 

  #No 2's or 3's test:
  #Probability to miss getting a 3 somewhere
  p_func = prob_to_miss_targets(jnp.array([1,1,0,0,0,0,0,0,0,0,0]))

  #TO miss getting a 2 or a 3 as possible sums you need either
  # No 1's OR (exactly one 1 AND no 2's)
  # Since these are disjoint we can just add the probabilities
  p_expected = (5/6)**4 + 4*(1/6)*(4/6)**3

  
  assert jnp.isclose(p_func,p_expected) 
  

  return True

assert prob_to_miss_targets_tests()

@jit
def cant_stop_bust_probability(runner_col,illegal_col):
  #Purpose:
  #  Compute the bust_probability if we were to roll again in Can't Stop
  #Input:
  #  runner_col = an array of shape (11,) of integers with the runner locations
  #  illegal_col = an array of shape (11,) of boolean with which columns are illegal to play in 
  #NOTE:
  #  We assume N_Max_Runners = 3 for this one!

  N_Max_Runners = 3

  runner_loc = (runner_col > 0)

  num_runners = jnp.sum(runner_loc) # calculate runners by counting non-zero indices
  target_columns = runner_col + (num_runners < N_Max_Runners)*(jnp.logical_not(illegal_col))
  
  #if num_runners >= N_Max_Runners:
  #  target_columns = runner_col
  #else:
  #  target_columns = jnp.logical_or(runner_col, 1-illegal_col)

  return prob_to_miss_targets(target_columns) # calculate the chance to miss playable columns

## The AI itself

In [29]:
w = jnp.ones(23)
@jit
def my_w_AI(active_player_index, player_col_state, choices, legal, N_COL_TO_WIN = 5, N_MAX_RUNNERS = 3):
  '''An AI function using some weights w which are assumed to be a global variable'''

  player_N_col_claimed = calculate_player_N_col_claimed(player_col_state)
  illegal_col = calculate_illegal_col(player_col_state)

  best_choice_index = -1
  best_choice_val = -1

  #This passes all the choices into the value function at once
  all_vals = v_func(w,active_player_index, player_col_state, player_N_col_claimed, illegal_col, choices)
  
  #Take the maximum over all the ones that are legal!
  best_choice_index = jnp.argmax(all_vals*legal)

 

  #Compute the value for rolling again and staying from that state.
  runner_col_state = choices[best_choice_index]
  q_roll_again_val = q_roll_again(w,active_player_index, player_col_state, player_N_col_claimed, illegal_col, runner_col_state)
  q_stay_val = q_stay(w,active_player_index, player_col_state, player_N_col_claimed, illegal_col, runner_col_state)


  return best_choice_index, q_roll_again_val > q_stay_val

In [30]:
@jit
def v_func(w,active_player_index, player_col_state, player_N_col_claimed, illegal_col, runner_col_state):
  '''Value function for the AI'''
  #Divide the weights w into three categories
  runner_w = w[0:11]
  location_w = w[11:22]
  N_col_claimed_w = w[22]

  #Note that if runner_col_state is a vector, score will be broadcast to that shape
  score = jnp.inner(runner_w, runner_col_state)
  
  player_col_diff = (player_col_state[active_player_index] - player_col_state[1- active_player_index])*(1 - illegal_col)
  score -= jnp.inner(location_w, player_col_diff)
  
  N_col_claimed_diff = player_N_col_claimed[active_player_index] - player_N_col_claimed[1 - active_player_index]
  score += 10*N_col_claimed_w*N_col_claimed_diff #This one is rescaled to be 10 times bigger because its important!
  
  return jnn.sigmoid(score)

grad_v_func = jax.grad(v_func,0)
w = jnp.ones(23)

In [31]:
@jit
def q_roll_again(w,active_player_index, player_col_state, player_N_col_claimed, illegal_col, runner_col_state):
  zero_runner = jnp.zeros( 11 ,dtype=jnp.dtype('u1')) 
  bust_value = 1-v_func(w, 1-active_player_index, player_col_state, player_N_col_claimed, illegal_col, zero_runner) 

  N_runner = jnp.count_nonzero(runner_col_state)
  advance_runner = runner_col_state + (1/3)*(runner_col_state > 0) + (1/12)*(3-N_runner) #This is approximate!!!
  advance_value = v_func(w, active_player_index, player_col_state, player_N_col_claimed, illegal_col, advance_runner)

  p_bust = cant_stop_bust_probability(runner_col_state,illegal_col)

  return p_bust*bust_value + (1-p_bust)*advance_value

@jit
def q_stay(w,active_player_index, player_col_state, player_N_col_claimed, illegal_col, runner_col_state):

  updated_player_col_state = update_player_col_state(active_player_index,player_col_state,runner_col_state) 
  updated_player_N_col_claimed = calculate_player_N_col_claimed(updated_player_col_state)
  updated_illegal_col = calculate_illegal_col(updated_player_col_state)
  zero_runner = jnp.zeros( 11 ,dtype=jnp.dtype('u1')) 

  return 1-v_func(w, 1-active_player_index, updated_player_col_state, updated_player_N_col_claimed, updated_illegal_col, zero_runner)

grad_q_roll_again = jax.grad(q_roll_again,0)
grad_q_stay = jax.grad(q_stay,0)
w = jnp.ones(23)

In [34]:
w = jnp.ones(23)
#simulate_game([my_w_AI,dummyAI],True)
%timeit simulate_game([my_w_AI,dummyAI],False)

1 loop, best of 5: 181 ms per loop


In [37]:
w = jnp.ones(23)
play_N_games(1000,[my_w_AI,dummyAI])
#my_w_AI with these hand set weights beats the dummyAI ~88% of the time after 1000 trials

0 % done
10 % done
20 % done
30 % done
40 % done
50 % done
60 % done
70 % done
80 % done
90 % done
After  1000  winners are  [880 120]


# Running multiple games at once

Running one game at a time is a bit too slow to be super useable for training, so here we set up a simulation that runners multiple games at the same time. This leads to a huge speedup! The AI program must be setup to input multiple states at once.

In [39]:
@jit
def random_true_index(key, bool_array):
  return jrandom.choice(key, jnp.arange(jnp.shape(bool_array)[0]) , p= bool_array/jnp.sum(bool_array))

random_true_index_by_rows = jax.vmap(random_true_index, in_axes = (None,0), out_axes=0)

#Tests
key = jrandom.PRNGKey(int(time.time()))
b =  jnp.array([True, False,True,False])
print(jnp.arange(jnp.shape(b)[0]))
random_true_index(key, b )

bb = jnp.array([[True, False],[False,False],[True,False]])
print(jnp.shape(bb))
ans = random_true_index_by_rows(key, jnp.transpose(bb, (1,0)) )
print(ans, jnp.shape(ans))

[0 1 2 3]
(3, 2)
[0 0] (2,)


In [40]:
def dummyAI_parr(random_key, active_player_index, player_col_state, choices, legal, N_COL_TO_WIN = 3, N_MAX_RUNNERS = 3):
  '''A placeholder game AI that makes all choices randomly'''
  #Input:
  #  active_player_index = An int with whose turn it currently is (which player the AI is playing for)
  #  player_col_state = An int array of size (N_players, 11, N_PARR) showing how many entries REMAINING until column is claimed for each player
  #  choices = An array of size (9,11,N_PARR) with the 9 possible choices available to the player
  #  legal = An array of size (9,N_PARR) with whether or not each of the 9 choices are legal
  #   N_Col_To_Win, N_Max_Runners = Integers that can specify variants of the game rules 
  #Output: A tupl (choice_index, roll_again)
  #  1st entry: choice_index = An array of shape (N_PARR,) of integers in [0,8] with which choice is to be played
  #             (Note, you must make sure the index you play is legal!)
  #  2nd entry: roll_again = A boolean array of shape (N_PARR,) on whether or not to roll again 
  
  choice_index = random_true_index_by_rows(random_key, jnp.transpose(legal,(1,0)) ) #choose a random legal option!
  random_key, subkey =  jrandom.split(random_key)
  N_PARR = jnp.shape(legal)[-1]
  #print("N_PARR", N_PARR)
  roll_again =jrandom.randint(random_key, (N_PARR,), 0,2) #choose to roll again randomly!
  #print(roll_again)
  
  #w = jnp.ones(23)
  #print(v_func(w,active_player_index, player_col_state, player_N_col_claimed, illegal_col, runner_col_state))
  #print(grad_v_func(w,active_player_index, player_col_state, player_N_col_claimed, illegal_col, runner_col_state))

  return choice_index, roll_again #dummy AI picks the first choice and stops rolling again

In [41]:
def simulate_game_parr(random_key,Player_AI, Verbose = False, Starting_Player=0, N_PARR=100, N_PLAYERS=2, N_COL_TO_WIN=5, N_MAX_RUNNERS=3, PLAYER_COL_STATE_INIT=[3,5,7,9,11,13,11,9,7,5,3]):
  '''Run a simulation of the game Can't Stop! running N_PARR games in parrallel'''
  #Input:
  #   randome_key = jrandom key used for dice rolls
  #   Player_AI = List with the functions for player AIs
  #   Verbose = whether or not to print out a play-by-play of the game
  #Output:
  #  An array of shape (N_players,) with a 1 at the player who won
  
  #Initialize game state
  player_col_state = jnp.tile(jnp.array(PLAYER_COL_STATE_INIT,dtype=jnp.dtype('i1')),(N_PARR, N_PLAYERS, 1))
  player_col_state = jnp.transpose(player_col_state, (1,2,0)) #Move it so the N_PARR dimension is LAST
  zero_runner = jnp.zeros( (11,N_PARR) ,dtype=jnp.dtype('u1')) 
  
  #Record which games are in progress (to make sure we don't mess with those games)
  game_in_progress = jnp.ones(N_PARR, dtype=bool)
  
  #Note that the player whose turn it is the SAME accross all games
  #This means its improtant to run the function more than once so there is no bias towards whoever plays first
  active_player_index = Starting_Player - 1 #random.randint(0,N_PLAYERS-1)
  
  #Main loop that goes until all games are over

  turn_iteration = 0
  
  while jnp.any(game_in_progress): #This will loop until the game ends 
    turn_iteration += 1
    
    #Update whose turn it is
    active_player_index = (active_player_index + 1) % N_PLAYERS
    
    if Verbose : print("Player ",active_player_index,":") 
    #if Verbose : print("--Player Column State: \n",player_col_state)

    #Reset runners and "busted"/"roll again" flags
    runner_col_state = zero_runner

    #roll_again_state and not_busted_state keep track of whether or not the player has
    #chosen to roll again and/or busted yet 
    #This is vector of length N_PARR and is only true for games still in progress
    roll_again_state = game_in_progress
    not_busted_state = game_in_progress 
    

    #Loop while player is chooising to rolling on their turn
    roll_iteration = 0
    while jnp.any( roll_again_state ):
      roll_iteration += 1
      #This represents a random dice roll for each N_PARR simulation 
      # (1296 = 6**4 is the number of possibilities for 4 6-sided dice)
      random_key, subkey =  jrandom.split(random_key)
      dice_num =  jrandom.randint(random_key, (N_PARR,) , 0,1296)
      
      if Verbose : print("----DiceNums: ",dice_num)      
      
      #Generate all 9 possible runner choices and whether or not they are legal
      runner_choices, runner_legal = generate_all_choices_and_legality(dice_num,player_col_state,runner_col_state, N_MAX_RUNNERS)
      
      #Update the busted state: you are only not busted if you have at least one legal choice
      any_legal_choices = jnp.any(runner_legal==True,axis=0)
      not_busted_state = jnp.logical_and(not_busted_state, any_legal_choices )
      
      #Send the choices to the AI to choose from
      #Note that we run this even for ones where we've already busted (we just make sure to do nothing with this data) 
      active_player_AI = Player_AI[active_player_index]
      random_key, subkey =  jrandom.split(random_key)
      choice_index, new_roll_again_state = active_player_AI(random_key, active_player_index, player_col_state, runner_choices, runner_legal, N_COL_TO_WIN, N_MAX_RUNNERS)
      
      #Find the runner position if they would advance according to the choices, choosing choice_index[i] for simulation number i
      new_runner_col_state = jnp.transpose(runner_choices[choice_index,:,jnp.arange(N_PARR)], (1,0) )

      #Update the runners to these new positions only if you were still rolling and not busted!
      # If you don't meet this criteria, then your roll doesnt count and runners stay where they are
      runner_col_state = jnp.where( jnp.logical_and(roll_again_state,not_busted_state), new_runner_col_state, runner_col_state)


      #In order to roll_again next round, three things must all happend: 
      #1. You are not busted
      #2. You chose to roll again last time 
      #3. You chose to roll again this time 
      roll_again_state = jnp.logical_and( jnp.logical_and( not_busted_state,roll_again_state), new_roll_again_state) #If you busted we set automatically set you to roll_again_state = False
      
      if Verbose : print("----Roll Iteration: ", roll_iteration,"\n", "Busted_State: ",busted_state,"Roll_Again_State: ",roll_again_state)
      
      
    
    #-----------------------
    #End of the players turn:
    #-----------------------
    
    #This line resets the runners of anyone who had busted to zero
    runner_col_state = runner_col_state * not_busted_state

    #Update the player positions!
    player_col_state = update_player_col_state(active_player_index,player_col_state,runner_col_state)
    
    if Verbose:
      [print("player_col_state Sim#",i,"\n",player_col_state[:,:,i]) for i in range(N_PARR)]
    player_N_col_claimed = calculate_player_N_col_claimed(player_col_state)
    
    game_in_progress = 1 - jnp.any(player_N_col_claimed >= N_COL_TO_WIN, axis=0)
    if Verbose: print("Turn #", turn_iteration, " Games in progress: ", jnp.sum(game_in_progress))
   
  #At the end of this loop, one player has won!
  if Verbose : 
    print("GAME OVER!") 
    print("Final board state: ", player_col_state)
    print("Final claimed column count: ", player_N_col_claimed)
    print("Winners: ", ( player_N_col_claimed >= N_COL_TO_WIN ))


  return jnp.sum( player_N_col_claimed >= N_COL_TO_WIN , axis=1)

In [42]:
key = jrandom.PRNGKey(int(time.time()))
N_PARR = 1000
simulate_game_parr(key,[dummyAI_parr,dummyAI_parr],False, 0, N_PARR)

DeviceArray([499, 501], dtype=int32)

In [ ]:
#On GPU can do:
# 1,000 in ~15sec
# 10,000 in ~25sec
# 100,000 in ~45sec
# 200,000 in ~1min15sec 
#  Efficiency starts to drop off around there, so it is better to run it multiple times if you need more than ~200,000 iters

key = jrandom.PRNGKey(int(time.time()))
N_PARR = 200000
N_RUNS = 5

total_runs = 0 #1000000
score = jnp.zeros(2)
for i in range(N_RUNS):
  print(i," of ", N_RUNS)
  total_runs += N_PARR
  key, _ = jrandom.split(key)
  score += simulate_game_parr(key,[dummyAI_parr,dummyAI_parr],False, 0, N_PARR)

print(100*score/total_runs)
#Seems that first turn advantage for the dummyAI is about ~1.8% (accurate to about ~0.1% over 1,000,000 runs)

0  of  5
